In [163]:
import pandas as pd
import numpy as np
import joblib
import os

In [164]:
train_01 = joblib.load('train_HA_feature/train_01.lz4')
train_02 = joblib.load('train_HA_feature/train_02.lz4')
train_03 = joblib.load('train_HA_feature/train_03.lz4')

test_01 = joblib.load('./test_HA_feature/test_01.lz4')
test_02 = joblib.load('./test_HA_feature/test_02.lz4')
test_03 = joblib.load('./test_HA_feature/test_03.lz4')
test_04 = joblib.load('./test_HA_feature/test_04.lz4')
test_05 = joblib.load('./test_HA_feature/test_05.lz4')

In [165]:
SCALE_DICT = dict(PCA_T2__mean=1/10, PCA_T2__mean_diff=10, PCA_T2__delta=1, PCA_T2__delta_diff=100)

In [166]:
def window_feature(df, has_label=False):
    
    data = df.copy()
    feature_cols = [col for col in data.columns if col not in ['CL', 'RULR']]
    data = data[feature_cols]
    # 缩放
    for key,val in SCALE_DICT.items():
        data[key] = data[key].values * val
        
    point_cnt = data.shape[0]-10+1
    x = [data.loc[i:i+9].values for i in range(point_cnt)]
    x = np.array(x)
    if has_label:
        y = df['RULR'].rolling(window=10, center=False, min_periods=10).mean().dropna().reset_index(drop=True).values
        return x, y 
    else:
        return x

In [167]:
# train
x_01, y_01 = window_feature(train_01, has_label=True)
x_02, y_02 = window_feature(train_02, has_label=True)
x_03, y_03 = window_feature(train_03, has_label=True)

In [168]:
x_train = np.vstack([x_01, x_02, x_03]).reshape(-1,10,4,1)
y_train = np.hstack([y_01, y_02, y_03]).reshape(-1,1)

In [169]:
# test
x_01 = window_feature(test_01, has_label=False)
x_02 = window_feature(test_02, has_label=False)
x_03 = window_feature(test_03, has_label=False)
x_04 = window_feature(test_04, has_label=False)
x_05 = window_feature(test_05, has_label=False)

In [170]:
x_test = np.vstack([x_01, x_02, x_03, x_04, x_05]).reshape(-1,10,4,1)

In [171]:
if not os.path.exists('./window_feature'):
    os.mkdir('./window_feature')
x_train.tofile('./window_feature/x_train')
y_train.tofile('./window_feature/y_train')
x_test.tofile('./window_feature/x_test')

In [174]:
np.exp(2)

7.38905609893065